### Imports

In [33]:
import pandas as pd
from decouple import config
import requests
import csv

### First Steps

Start with a small test sample of books, scanned using a barcode scanner

In [2]:
df = pd.read_clipboard(header = 0)

In [54]:
firstSearch = "0201129108"

In [7]:
# pulling in the authentication information from the env variable
apiKey = config('KEY')
apiUser = config('USER')

In [55]:
# making an initial request
payload = {'key' : apiKey}
r = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:{firstSearch}', params = payload)

In [56]:
r.json()

{'kind': 'books#volumes',
 'totalItems': 1,
 'items': [{'kind': 'books#volume',
   'id': 'DiLvAAAAMAAJ',
   'etag': 'XT61pc7PHEI',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/DiLvAAAAMAAJ',
   'volumeInfo': {'title': 'A Concrete Approach to Mathematical Modelling',
    'authors': ['Mike Mesterton-Gibbons'],
    'publisher': 'Addison Wesley Publishing Company',
    'publishedDate': '1989',
    'description': 'Critical praise for A Concrete Approach to Mathematical Modelling "...a treasure house of material for students and teachers alike...can be dipped into regularly for inspiration and ideas. It deserves to become a classic."-London Times Higher Education Supplement "The author succeeds in his goal of serving the needs of the undergraduate population who want to see mathematics in action, and the mathematics used is extensive and provoking."-SIAM Review "Each chapter discusses a wealth of examples ranging from old standards...to novelty ... Each model is developed crit

In [30]:
r.json()['items'][0]['volumeInfo']['title']

'The Greatest Show on Earth'

In [52]:
def getDeets(isbn):
    row = {}
    row['ISBN'] = isbn
    payload = {'key' : apiKey}
    r = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}', params = payload)
    info = r.json()['items'][0]['volumeInfo']
    try:
        row['title'] = info['title']
        if 'subtitle' in info:
            row['subtitle'] = info['subtitle']
        else:
            row['subtitle'] = 'NA'
        row['authors'] = info['authors']
        row['pageCount'] = info['pageCount']
        row['categories'] = info['categories']
        row['averageRating'] = info['averageRating']
        row['ratingsCount'] =  info['ratingsCount']
        row['publishedDate'] = info['publishedDate']
        row['publisher'] = info['publisher']
        row['description'] = info['description']
    except Exception as e:
        print(f"Problem with gathering info: \n{e}" )
        
    for k, v in row.items():
        print(f"{k} : {v}")
    print("\n")
    
    # now print to csv
    try:
        with open('output.csv', "a", newline='') as csvFile:
            writer = csv.writer(csvFile, delimiter=',')
            writer.writerow([row['ISBN'], 
                             row['title'], 
                             row['subtitle'], 
                             row['authors'], 
                             row['pageCount'], 
                             row['categories'], 
                             row['averageRating'], 
                             row['ratingsCount'],
                             row['publishedDate'], 
                             row['publisher'], 
                             row['description']])
    except Exception as e:
        print(f"Problem with writing to file: \n{e}" )

In [53]:
for eachISBN in df['ISBN'].to_list()[1:]:
    getDeets(eachISBN)

ISBN : 9780151005574
title : Looking for Spinoza
subtitle : Joy, Sorrow, and the Feeling Brain
authors : ['Antonio R. Damasio']
pageCount : 355
categories : ['Philosophy']
averageRating : 3.5
ratingsCount : 6
publishedDate : 2003
publisher : Houghton Mifflin
description : Investigates the cerebral mechanisms behind emotions and feelings to explain the role between emotion, survival, and cultural accomplishment.


ISBN : 9781473620919
title : Thing Explainer
subtitle : Complicated Stuff in Simple Words
authors : ['Randall Munroe']
pageCount : 61
categories : ['Humor']
averageRating : 3.5
ratingsCount : 8
publishedDate : 2015-11-19
publisher : John Murray
description : From the No. 1 bestselling author of What If? - the man who created xkcd and explained the laws of science with cartoons - comes a series of brilliantly simple diagrams ('blueprints' if you want to be complicated about it) that show how important things work: from the nuclear bomb to the biro. It's good to know what the pa